In [ ]:
# ! pip install -r ../../requirements.txt

In [ ]:
import dotenv

dotenv.load_dotenv('.env')

In [ ]:
from langfuse import Langfuse
 
langfuse = Langfuse()
langfuse_prompt = langfuse.get_prompt("rag-base-prompt-with-context", cache_ttl_seconds=0)
print(langfuse_prompt.prompt)

In [ ]:
online_prompt = langfuse_prompt.get_langchain_prompt()
online_prompt

In [ ]:
langfuse_prompt.config

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Artificial intelligence", load_max_docs=2, doc_content_chars_max=1000).load()
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)
documents=text_splitter.split_documents(docs)
for doc in documents[:5]:
    print(doc.page_content, "\n")
print(len(documents))

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=langfuse_prompt.config["model"], temperature=langfuse_prompt.config["temperature"])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(online_prompt)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
from langfuse.callback import CallbackHandler
from langfuse.decorators import langfuse_context

langfuse_handler = CallbackHandler(
    session_id="langchain-test",
    user_id="kartik@gmail.com",
    metadata={"is_notebook": True},
    tags=["lang_check"]
)
# langfuse_handler = langfuse_context.get_current_langchain_handler()
print(rag_chain.invoke("what is AI ?", config={"callbacks": [langfuse_handler]}))

# optional
langfuse_handler.flush()

# Model based Evaluation
- we can use available or a custom template
- can run evaluations based on filter criterias
- Finally we the scores are updated

In [ ]:
"""
Evaluate the query's different text languages and the generation on a continuous scale from 0 to 1. A generation can be considered correct (Score: 1) if the generation language is the same as the query language.

Input:
Query: com'è il tempo in India?
Generation: The weather in India is hot and humid. Current dusty and partly cloudy conditions are expected to continue throughout the day.
Score: 0
Reasoning: The query was in Italian language but the generation was in English

Input:
Query: what is a galaxy?
Generation: A galaxy is a cluster of stars. notre soleil est un début
Score: 0.4
Reasoning: The query was in English but the generation was partially in English and partially in French

Input:
Query: {{query}}
Generation: {{generation}}

Think step by step.
"""

"""
Score between 0 and 1
Give a one word output of the query language detected
lang_check
"""

In [ ]:
langfuse_handler = CallbackHandler(
    session_id="model-based-eval-check",
    user_id="kartik@gmail.com",
    metadata={"is_notebook": True},
    tags = ["lang_check"]
)
rag_chain.invoke("cosa è AI ?", config={"callbacks": [langfuse_handler]})